<a href="https://colab.research.google.com/github/errazuriz/centinela/blob/master/1_limpieza_hogares_y_direcciones_(20200622).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 143kB 4.6MB/s 


In [ ]:
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
import xlsxwriter
import os
from google.colab import files

In [ ]:
url_centinela = "https://covid.renca.cl/vapps/Admin/Case/Case?workEffortId="

In [ ]:
base_dir = "/content/drive/My Drive/@CORONAVIRUS/CORONAVIRUS COMPARTIDO/LIMPIEZA DATOS CENTINELA"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Carga de listado de cajas entregadas

In [ ]:
folder = "input"
timpestamp_simpliroute = "20200620_12_24"
filename =  "Reporte de Visitas SimpliRoute({}).xlsx".format(timpestamp_simpliroute)
path_input_simpliroute = os.path.join(base_dir,folder,filename)

In [ ]:
path_input_simpliroute

'/content/drive/My Drive/@CORONAVIRUS/CORONAVIRUS COMPARTIDO/LIMPIEZA DATOS CENTINELA/input/Reporte de Visitas SimpliRoute(20200620_12_24).xlsx'

In [ ]:
simpliroute = pd.read_excel(path_input_simpliroute, sheet_name="Sheet1")
simpliroute.columns =[column.replace(" ", "_") for column in simpliroute.columns]
filtrado_simpliroute = simpliroute.query('(Vehículo == ["Móvil M","Movil N","Movil O"] or (Vehículo == ["Movil C","Movil D","Movil F"] and Fecha_planificada == datetime(2020,6,9)))')

In [ ]:
filtrado_simpliroute['semana'] = filtrado_simpliroute.Fecha_planificada.dt.week

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
filtrado_simpliroute.Fecha_planificada = filtrado_simpliroute.Fecha_planificada.dt.date

In [ ]:
folder = "output"
timestamp_simpliroute = "20200620_12_24"
filename =  "Reporte de Visitas SimpliRoute({})_PROCESADO.xlsx".format(timestamp_simpliroute)
path_output_simpliroute = os.path.join(base_dir,folder,filename)
filtrado_simpliroute.to_excel(path_output_simpliroute,index=False)
files.download(path_output_simpliroute)

In [ ]:
filtrado_simpliroute.Estado.value_counts(dropna=False)

completed    2328
failed        312
pending       135
Name: Estado, dtype: int64

In [ ]:
pd.pivot_table(filtrado_simpliroute,index=["semana"],values=["Tracking_ID"],columns=["Estado"],aggfunc=pd.Series.nunique,margins=True)

Tracking_ID                     
Estado   completed failed pending   All
semana                                 
23             375     60      29   464
24            1105    142      98  1345
25             848    110       8   966
All           2328    312     135  2775

In [ ]:
pd.pivot_table(filtrado_simpliroute.query('Estado=="completed" & semana==24'),index=["Fecha_planificada"],values=["Tracking_ID"],columns=["Vehículo"],aggfunc=pd.Series.nunique,margins=True)

Tracking_ID                                              
Vehículo              Movil C Movil D Movil F Movil N Movil O Móvil M   All
Fecha_planificada                                                          
2020-06-08                NaN     NaN     NaN    65.0    57.0    66.0   188
2020-06-09               68.0    64.0    61.0    73.0    62.0    72.0   400
2020-06-10                NaN     NaN     NaN    70.0     NaN    76.0   146
2020-06-11                NaN     NaN     NaN    62.0    61.0    61.0   184
2020-06-12                NaN     NaN     NaN    60.0    63.0    64.0   187
All                      68.0    64.0    61.0   330.0   243.0   339.0  1105

In [ ]:
filtrado_simpliroute['Id_de_referencia'] = filtrado_simpliroute['Id_de_referencia'].str.replace(r'.', '')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
filtrado_simpliroute.loc[filtrado_simpliroute.Id_de_referencia.isna()]

,Tracking_ID,Id_de_referencia,Fecha_planificada,Conductor,Vehículo,Título,Dirección,ETA,ETD,Checkin,Checkout,Tiempo_de_servicio_estimado,Tiempo_de_servicio_real,Anticipo,Atraso,Latitud,Longitud,Checkout_latitud,Checkout_longitud,Carga,Carga_2,Carga_3,Estado,Comentarios,Observaciones,Inicio_de_ventana_horaria_1,Final_de_ventana_horaria_1_,Inicio_de_ventana_horaria_2,Final_de_ventana_horaria_2_,Habilidades_requeridas,Habilidades_opcionales,Notas,Persona_de_contacto,Teléfono_de_contacto,Correo_electrónico_de_contacto,ID_de_ruta
1723,SR91113983015614,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"RÍO CAUTIN 4867, RENCA, REGION METROPOLITANA",09:07:00,09:17:00,2020-06-02 14:18:25,2020-06-02 14:33:17,00:10:00,00:14:51,00:00:00,05:16:00,-33.397732,-70.721665,-33.397773,-70.721407,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a60-4f70-a271-2f8b7c6f09cb
1724,SR91113983112382,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"RÍO HUASCO 4885, RENCA, REGION METROPOLITANA",09:18:00,09:28:00,2020-06-02 14:37:16,2020-06-02 14:37:39,00:10:00,00:00:22,00:00:00,05:09:00,-33.395968,-70.722820,-33.395985,-70.722833,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a60-4f70-a271-2f8b7c6f09cb
1725,SR59111398323047,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"RIO RAHUE 5173 BLOCK C1 DP 24, RENCA, REGION M...",09:30:00,09:40:00,2020-06-02 14:37:59,2020-06-02 14:52:14,00:10:00,00:14:15,00:00:00,05:12:00,-33.397695,-70.725376,-33.397443,-70.725370,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a60-4f70-a271-2f8b7c6f09cb
1726,SR91113983287613,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"NUEVA UNO 1473, RENCA, REGION METROPOLITANA",09:43:00,09:53:00,NaT,2020-06-02 15:00:27,00:10:00,00:00:00,00:00:00,05:07:00,-33.403154,-70.723754,-33.403279,-70.723645,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a60-4f70-a271-2f8b7c6f09cb
1727,SR91113983372563,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"Pasaje Cuatro 1439, Renca, Región Metropolitan...",09:57:00,10:07:00,2020-06-02 14:27:38,2020-06-02 15:06:11,00:10:00,00:38:33,00:00:00,04:59:00,-33.403204,-70.717290,-33.403098,-70.717356,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a60-4f70-a271-2f8b7c6f09cb
1728,SR91113983438500,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"RAFAEL EGAÑA 3174, RENCA, REGION METROPOLITANA",10:58:00,11:08:00,NaT,2020-06-02 15:19:22,00:10:00,00:00:00,00:00:00,04:11:00,-33.414468,-70.689432,-33.414410,-70.689393,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a60-4f70-a271-2f8b7c6f09cb
1729,SR91113983537430,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"PASAJE 7 ,1522, RENCA, REGION METROPOLITANA",11:14:00,11:24:00,NaT,2020-06-02 15:28:07,00:10:00,00:00:00,00:00:00,04:04:00,-33.405726,-70.694322,-33.405596,-70.694104,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a60-4f70-a271-2f8b7c6f09cb
1730,SR91113983599976,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"PJE DOS SUR 3841 BLOCK 10 DP 34, RENCA, REGION...",11:30:00,11:40:00,2020-06-02 14:23:18,2020-06-02 15:47:41,00:10:00,01:24:23,00:00:00,04:07:00,-33.404452,-70.697806,-33.404656,-70.698143,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a60-4f70-a271-2f8b7c6f09cb
1731,SR91113983688465,NaN,2020-06-02,Denisse Huichacura,Movil N,Visita sin título,"QUELTEHUE 1626, RENCA, REGION METROPOLITANA",11:43:00,11:53:00,2020-06-02 14:22:56,2020-06-02 15:55:18,00:10:00,01:32:22,00:00:00,04:02:00,-33.403050,-70.699037,-33.402898,-70.699014,0,0,0,completed,NaN,Exitoso,00:00:00,23:59:00,23:59:00,23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,c88c9647-3a6

In [ ]:
td = pd.pivot_table(filtrado_simpliroute,index=["Id_de_referencia"],values=["Tracking_ID"],columns=["Estado"],aggfunc=pd.Series.nunique,margins=True)

In [ ]:
td.columns = td.columns.get_level_values(1)

# Carga de datos desde Centinela

In [ ]:
timpestamp_centinela = "20200618_11_08" 
path_input_centinela = "/content/drive/My Drive/@CORONAVIRUS/CORONAVIRUS COMPARTIDO/LIMPIEZA DATOS CENTINELA/input/CaseByRoleList({}).xlsx".format(timpestamp_centinela)
centinela = pd.read_excel(path_input_centinela)

In [ ]:
df = centinela
df.columns =[column.replace(" ", "_") for column in df.columns]

## Join entre Centinela y Simpliroute

In [ ]:
df = df.merge(td,left_on='Doc_Identidad', right_on='Id_de_referencia', how='left')

In [ ]:
path_output = "/content/drive/My Drive/@CORONAVIRUS/CORONAVIRUS COMPARTIDO/LIMPIEZA DATOS CENTINELA/output/centinela/Centinela_combinado_con_simpliroute({}).xlsx".format(timpestamp_centinela)
df.to_excel(path_output,index=False)

# Limpieza Centinela
## Tipos de datos

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11157 entries, 0 to 11156
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID_del_Caso                             11157 non-null  int64         
 1   Tipo_de_Caso                            8909 non-null   object        
 2   Fecha_de_Inicio_Caso                    11154 non-null  datetime64[ns]
 3   Paciente                                11157 non-null  object        
 4   Tipo_de_Documento                       11131 non-null  object        
 5   Doc_Identidad                           11131 non-null  object        
 6   Género_del_Paciente                     11157 non-null  object        
 7   Fecha_Último_Test                       6800 non-null   datetime64[ns]
 8   Fecha_Últimos_Resultados                5880 non-null   datetime64[ns]
 9   Resultado_Último_Test                   6589 non-n

In [ ]:
df.iloc[0,:]

ID_del_Caso                                                                 100001
Tipo_de_Caso                                                                   NaN
Fecha_de_Inicio_Caso                                           2020-03-10 00:00:00
Paciente                                                Camilo Recabarren Gonzalez
Tipo_de_Documento                                       Rol Único Tributario (RUT)
Doc_Identidad                                                           19092371-1
Género_del_Paciente                                                      Masculino
Fecha_Último_Test                                              2020-03-10 00:00:00
Fecha_Últimos_Resultados                                       2020-03-10 00:00:00
Resultado_Último_Test                                                     Negativo
Tipo_de_Seguimiento                       Distanciamiento Social (Sin Seguimiento)
¿Es_el_caso_índice?                                                             No
Últi

In [ ]:
df.select_dtypes(include ='datetime64') = df.select_dtypes(include ='datetime64').apply(lambda x: pd.to_datetime(x).dt.date)

SyntaxError: ignored

In [ ]:
df.select_dtypes(include ='datetime64')

,Fecha_de_Inicio_Caso,Fecha_Último_Test,Fecha_Últimos_Resultados,First_Test_Date,Fecha_primer_test_positivo,First_Positive_Change_Date,Creation_Timestamp,Fecha_Último_Cambio_Tipo_Seguimiento,Last_Case_Update_Timestamp,Last_Active_Quarantine_From_Date
0,2020-03-10,2020-03-10,2020-03-10,2020-03-10,NaT,NaT,2020-04-30 21:52:02.025,2020-05-01 01:33:58.036,2020-04-30 21:52:16.399,NaT
1,2020-03-12,2020-03-12,2020-03-12,2020-03-12,NaT,NaT,2020-04-30 21:52:02.047,2020-05-01 01:33:57.968,2020-04-30 21:52:16.399,NaT
2,2020-03-12,2020-03-12,2020-03-12,2020-03-12,NaT,NaT,2020-04-30 21:52:02.072,2020-05-01 01:33:57.889,2020-04-30 21:52:16.399,NaT
3,2020-03-13,2020-03-13,2020-03-13,2020-03-13,NaT,NaT,2020-04-30 21:52:02.084,2020-05-01 01:33:58.194,2020-04-30 21:52:16.399,NaT
4,2020-03-15,2020-03-15,2020-03-15,2020-03-15,NaT,NaT,2020-04-30 21:52:02.089,2020-05-01 01:33:58.210,2020-04-30 21:52:16.399,NaT
...,...,...,...,...,...,...,...,...,...,...
11152,2020-06-17,2020-06-17,NaT,2020-06-17,NaT,NaT,2020-06-18 10:54:34.919,2020-06-18 10:54:34.919,NaT,2020-06-17
11153,2020-06-18,NaT,NaT,NaT,NaT,NaT,2020-06-18 10:54:41.845,2020-06-18 10:54:41.845,NaT,NaT
11154,2020-06-17,2020-06-17,NaT,2020-06-17,NaT,NaT,2020-06-18 10:57:53.783,2020-06-18 10:57:53.783,NaT,2020-06-17
11155,2020-06-18,NaT,NaT,NaT,NaT,NaT,2020-06-18 10:59:59.033,2020-06-18 10:59:59.033,NaT,NaT


In [ ]:
df.Pending_Test_Date = df.Pending_Test_Date.dt.date
df.Creation_Timestamp = df.Creation_Timestamp.dt.date

## EDA Centinela

In [ ]:
#Cuantos Id unicos y cuantos ID. Si son iguales, no hay ID duplicados en la base
print('|unicos:',len(df.ID_del_Caso.unique()),'|todos:',len(df.ID_del_Caso),'|no hay duplicados:',len(df.ID_del_Caso.unique()) == len(df.ID_del_Caso))

|unicos: 11157 |todos: 11157 |no hay duplicados: True


In [ ]:
df.columns

Index(['ID_del_Caso', 'Tipo_de_Caso', 'Fecha_de_Inicio_Caso', 'Paciente',
       'Tipo_de_Documento', 'Doc_Identidad', 'Género_del_Paciente',
       'Fecha_Último_Test', 'Fecha_Últimos_Resultados',
       'Resultado_Último_Test', 'Tipo_de_Seguimiento', '¿Es_el_caso_índice?',
       'Último_Estado_de_Salud', 'Institución_de_Ingreso',
       'Equipo_de_Seguimiento', 'Funcionario_Asignado', 'Dirección',
       'Block_/_Depto_/_Casa', 'Comuna', 'Barrio', 'Macrozona', 'Latitud',
       'Longitud', 'First_Test_Date', 'First_Test_Result', 'Pending_Test_Date',
       'Fecha_primer_test_positivo', 'First_Positive_Test_Result',
       'First_Positive_Change_Date', 'Creation_Timestamp',
       'Persona_que_creó_el_caso', 'Fecha_Último_Cambio_Tipo_Seguimiento',
       'Persona_Último_Cambio_Tipo_Seguimiento', 'Last_Case_Update_Timestamp',
       'Last_Active_Quarantine_From_Date', 'Quarantine_In_Private_Address',
       'Telecom_Number', 'Correo_Electrónico', 'completed', 'failed',
       'pending

# Auditoria


## Deteccion de missing e incompletos

1.   Tipos de seguimiento
2.   Tipos de caso
3.   Direcciones incompletas
4.   Telefonos incompletos 




In [ ]:
# TIPOS DE SEGUIMIENTO = MISSING
missing_tipos_de_seguimiento = df[df.Tipo_de_Seguimiento.isnull()]
print("Filas con Tipos de Seguimiento = MISSING:",len(missing_tipos_de_seguimiento))

Filas con Tipos de Seguimiento = MISSING: 205


In [ ]:
# TIPOS DE SEGUIMIENTO = MISSING ABIERTOS POR SEMANA
missing_tipos_de_seguimiento.Creation_Timestamp.dt.week.value_counts(dropna=False).sort_index(ascending=False)

25    22
24    50
23    26
22     6
21    13
20    45
19    25
18    18
Name: Creation_Timestamp, dtype: int64

In [ ]:
# TIPOS DE CASO = MISSING
missing_tipos_de_caso = df[df.Tipo_de_Caso.isnull()]
print("Filas con Tipos de Caso = MISSING:",len(missing_tipos_de_caso))

Filas con Tipos de Caso = MISSING: 2248


In [ ]:
folder = "output"

In [ ]:
filename = "1_Auditoria_MISSING_VALUES_Centinela_({}).xlsx".format(timpestamp_centinela)

In [ ]:
os.path.join(base_dir,folder,filename)

'/content/drive/My Drive/@CORONAVIRUS/CORONAVIRUS COMPARTIDO/LIMPIEZA DATOS CENTINELA/output/1_Auditoria_MISSING_VALUES_Centinela_(20200618_11_08).xlsx'

### Exportar planillas excel con missings

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
complete_filename = os.path.join(base_dir,folder,filename)
writer = pd.ExcelWriter(complete_filename, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
missing_tipos_de_seguimiento.to_excel(writer, sheet_name='1_missing_tipos_seg')
missing_tipos_de_caso.to_excel(writer, sheet_name='2_missing_tipos_caso')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
files.download(complete_filename)

## Deteccion de anomalias
### Valores fuera de rango

# Primer test: Tipo de seguimiento

Reglas:
- No se puede estar "En Cuarentena" si tu fecha de creacion es mas de 14 dias en el pasado, excepto si tienen test pendiente
- Tu cuarentena expira 14 dias despues del fisrt_positive_test_result

In [ ]:
df['creation_week'] = df.Creation_Timestamp.dt.week

In [ ]:
df['creation_day'] = df.Creation_Timestamp.dt.date

In [ ]:
dt.date.today()

datetime.date(2020, 6, 18)

In [ ]:
df['dias_desde_creacion'] = dt.date.today()-df.Creation_Timestamp.dt.date

In [ ]:
df['dias_desde_creacion'] = df['dias_desde_creacion'].dt.days

In [ ]:
df.First_Positive_Test_Result

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
11152    NaN
11153    NaN
11154    NaN
11155    NaN
11156    NaN
Name: First_Positive_Test_Result, Length: 11157, dtype: object

In [ ]:
df['first_positive_result_day'] = df.First_Positive_Test_Result.dt.date
df['dias_desde_first_positive_test_result'] = datetime.date.today()-df.First_Positive_Test_Result.dt.date
df['dias_desde_first_positive_test_result'] = df['dias_desde_first_positive_test_result'].dt.days

AttributeError: ignored

# Test pendientes

In [ ]:
df.Pending_Test_Date.value_counts(dropna=False)

NaT                        10149
2020-06-17 00:00:00.000      166
2020-06-16 00:00:00.000      140
2020-06-15 00:00:00.000      137
2020-06-14 00:00:00.000       57
2020-05-20 00:00:00.000       38
2020-06-03 00:00:00.000       37
2020-06-04 00:00:00.000       36
2020-06-02 00:00:00.000       35
2020-06-13 00:00:00.000       34
2020-05-18 00:00:00.000       27
2020-05-24 00:00:00.000       24
2020-05-21 00:00:00.000       24
2020-05-19 00:00:00.000       24
2020-05-25 00:00:00.000       20
2020-05-16 00:00:00.000       20
2020-06-01 00:00:00.000       19
2020-05-31 00:00:00.000       17
2020-06-12 00:00:00.000       12
2020-05-27 00:00:00.000       12
2020-05-29 00:00:00.000       10
2020-06-18 00:00:00.000       10
2020-05-30 00:00:00.000        9
2020-05-28 00:00:00.000        9
2020-05-23 00:00:00.000        9
2020-05-22 00:00:00.000        7
2020-04-29 00:00:00.000        6
2020-05-06 00:00:00.000        6
2020-05-15 00:00:00.000        6
2020-05-26 00:00:00.000        6
2020-05-05

In [ ]:
df.Pending_Test_Date.notnull().sum()

1008

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11157 entries, 0 to 11156
Data columns (total 41 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID_del_Caso                             11157 non-null  int64         
 1   Tipo_de_Caso                            8909 non-null   object        
 2   Fecha_de_Inicio_Caso                    11154 non-null  datetime64[ns]
 3   Paciente                                11157 non-null  object        
 4   Tipo_de_Documento                       11131 non-null  object        
 5   Doc_Identidad                           11131 non-null  object        
 6   Género_del_Paciente                     11157 non-null  object        
 7   Fecha_Último_Test                       6800 non-null   datetime64[ns]
 8   Fecha_Últimos_Resultados                5880 non-null   datetime64[ns]
 9   Resultado_Último_Test                   6589 non-n

In [ ]:
pd.pivot_table(df,index=["Tipo_de_Seguimiento"],values=["ID_del_Caso"],columns=["creation_week"],aggfunc=pd.Series.nunique,margins=True)

ID_del_Caso         ...               
creation_week                                     18     19  ...      25    All
Tipo_de_Seguimiento                                          ...               
Derivado a Hotel Sanitario                       NaN    NaN  ...     NaN     20
Derivado a Urgencia de Hospital                  2.0    NaN  ...     8.0     39
Derivado a otra Comuna (Sin Seguimiento)        17.0    5.0  ...     7.0     91
Desestimado (Sin Seguimiento)                  331.0   37.0  ...     6.0    691
Distanciamiento Social (Sin Seguimiento)       487.0   83.0  ...    94.0   2216
En Cuarentena                                   48.0   60.0  ...  1085.0   4754
Fallecido (Sin Seguimiento)                      9.0    8.0  ...     5.0    140
Hospitalizado                                    4.0    4.0  ...    10.0    116
Recuperado (Sin Seguimiento)                   322.0  246.0  ...    19.0   2629
Seguimiento Post-Cuarentena                      2.0    7.0  ...     4.0    197
Sin Información                                  7.0    1.0  ...    23.0     59
All                                           1229.0  451.0  ...  1261.0  10952

[12 rows x 9 columns]

In [ ]:
df.query('Tipo_de_Seguimiento == "En Cuarentena" and creation_week==24')

,ID_del_Caso,Tipo_de_Caso,Fecha_de_Inicio_Caso,Paciente,Tipo_de_Documento,Doc_Identidad,Género_del_Paciente,Fecha_Último_Test,Fecha_Últimos_Resultados,Resultado_Último_Test,Tipo_de_Seguimiento,¿Es_el_caso_índice?,Último_Estado_de_Salud,Institución_de_Ingreso,Equipo_de_Seguimiento,Funcionario_Asignado,Dirección,Block_/_Depto_/_Casa,Comuna,Barrio,Macrozona,Latitud,Longitud,First_Test_Date,First_Test_Result,Pending_Test_Date,Fecha_primer_test_positivo,First_Positive_Test_Result,First_Positive_Change_Date,Creation_Timestamp,Persona_que_creó_el_caso,Fecha_Último_Cambio_Tipo_Seguimiento,Persona_Último_Cambio_Tipo_Seguimiento,Last_Case_Update_Timestamp,Last_Active_Quarantine_From_Date,Quarantine_In_Private_Address,Telecom_Number,Correo_Electrónico,completed,failed,pending,All,creation_week,creation_day,dias_desde_creacion
7962,112987,NaN,2020-06-05,Abraham Colihuinca Leuman,Rol Único Tributario (RUT),12183149-K,Masculino,2020-06-05,2020-06-14,Negativo,En Cuarentena,Sí,Sano,CESFAM H. Urzúa,Unidad CESFAM H. Urzúa,NaN,"LOS ARTESANOS 1165, Renca, Chile",block b /dpto 112,Renca,NaN,NaN,NaN,NaN,2020-06-05,Negativo,NaT,NaT,NaN,NaT,2020-06-08 08:55:41.240,Estefany Vivar,2020-06-08 08:55:41.240,Estefany Vivar,2020-06-15 16:41:06.970,NaT,NaN,56 953150514,NaN,NaN,NaN,NaN,NaN,24,2020-06-08,10
7965,112990,Contacto de Alto Riesgo,2020-06-04,Denisse Arroyo Vidal,Rol Único Tributario (RUT),18062779-0,Femenino,2020-06-04,NaT,No Sabemos,En Cuarentena,No,Leve,CESFAM H. Urzúa,Unidad CESFAM H. Urzúa,NaN,"Eros 7142, Renca, Chile",NaN,Renca,NaN,NaN,-33.406431,-70.743149,2020-06-04,No Sabemos,2020-06-04,NaT,NaN,NaT,2020-06-08 09:15:14.275,Estefany Vivar,2020-06-08 09:15:14.275,Estefany Vivar,2020-06-15 16:52:39.872,NaT,NaN,56 232477636,NaN,1.0,NaN,NaN,1.0,24,2020-06-08,10
7968,112993,Contacto de Alto Riesgo,2020-06-08,Ana Calderon Fabres,Rol Único Tributario (RUT),7101707-9,Femenino,NaT,NaT,NaN,En Cuarentena,No,Leve,CESFAM Renca,Unidad CESFAM Renca,Lucia Tong,"Pasaje Once 1487, Renca, Chile",NaN,Renca,NaN,NaN,-33.405334,-70.696374,NaT,NaN,NaT,NaT,NaN,NaT,2020-06-08 09:34:12.561,Lucia Tong,2020-06-08 09:34:12.561,Lucia Tong,2020-06-09 12:45:16.028,NaT,NaN,56 976962581,NaN,NaN,NaN,NaN,NaN,24,2020-06-08,10
7970,112995,Contacto de Alto Riesgo,2020-06-07,Jose Chavez Puentes,Rol Único Tributario (RUT),16715489-1,Masculino,NaT,NaT,NaN,En Cuarentena,No,Leve,CESFAM H. Urzúa,Unidad CESFAM H. Urzúa,NaN,"Zeus 952, Renca, Chile",NaN,Renca,NaN,NaN,-33.410365,-70.740802,NaT,NaN,NaT,NaT,NaN,NaT,2020-06-08 09:47:41.910,Estefany Vivar,2020-06-08 09:47:41.910,Estefany Vivar,2020-06-16 10:11:25.523,NaT,NaN,56 971075303,NaN,NaN,NaN,NaN,NaN,24,2020-06-08,10
7971,112996,Contacto de Alto Riesgo,2020-06-07,Constanza Chavez Ruiz,Rol Único Tributario (RUT),23441965-K,Femenino,NaT,NaT,NaN,En Cuarentena,No,Sano,CESFAM H. Urzúa,Unidad CESFAM H. Urzúa,NaN,"Zeus 952, Renca, Chile",NaN,Renca,NaN,NaN,-33.410365,-70.740802,NaT,NaN,NaT,NaT,NaN,NaT,2020-06-08 09:52:49.039,Estefany Vivar,2020-06-08 09:52:49.039,Estefany Vivar,2020-06-08 09:53:21.105,NaT,NaN,56 971075303,NaN,NaN,NaN,NaN,NaN,24,2020-06-08,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,115177,Comprobado positivo (PCR positiva),2020-06-14,Moisés Felipe Baez López,Rol Único Tributario (RUT),19885262-7,Masculino,2020-06-12,2020-06-14,Positivo,En Cuarentena,Sí,Sin Información,SAR Renca,Unidad CESFAM H. Urzúa,NaN,"Hércules 919, Renca, Chile",NaN,Renca,NaN,NaN,-33.410970,-70.745221,2020-06-12,Positivo,NaT,2020-06-12,Positivo,2020-06-14 18:09:13.469,2020-06-14 18:09:13.454,Constanza del Rio Gomez,2020-06-14 18:09:13.454,Constanza del Rio Gomez,NaT,2020-06-12,True,56 930115987,NaN,1.0,NaN,NaN,1.0,24,2020-06-14,4
9910,115178,Comprobado positivo (PCR positiva),2020-06-14,Oscar Andres Muñoz Bravo,Rol Único Tributario (RUT),17171007-3,Masculino,2020-06-12,2020-06-14,Positivo,En Cuarentena,Sí,Asintomá

In [ ]:
pd.pivot_table(df.query('Tipo_de_Seguimiento == "En Cuarentena" and creation_week==24'),index=["Tipo_de_Seguimiento"],values=["ID_del_Caso"],columns=["Último_Estado_de_Salud"],aggfunc=pd.Series.nunique,margins=True)

ID_del_Caso                 ...                           
Último_Estado_de_Salud Asintomático Fallecido Leve  ... Sano Sin Información   All
Tipo_de_Seguimiento                                 ...                           
En Cuarentena                   395         1  440  ...  100             357  1369
All                             395         1  440  ...  100             357  1369

[2 rows x 9 columns]

In [ ]:
#casos con cuarentena de mas de 14 dias
df.query('Tipo_de_Seguimiento == "En Cuarentena" and dias_desde_creacion > 14')

,ID_del_Caso,Tipo_de_Caso,Fecha_de_Inicio_Caso,Paciente,Tipo_de_Documento,Doc_Identidad,Género_del_Paciente,Fecha_Último_Test,Fecha_Últimos_Resultados,Resultado_Último_Test,Tipo_de_Seguimiento,¿Es_el_caso_índice?,Último_Estado_de_Salud,Institución_de_Ingreso,Equipo_de_Seguimiento,Funcionario_Asignado,Dirección,Block_/_Depto_/_Casa,Comuna,Barrio,Macrozona,Latitud,Longitud,First_Test_Date,First_Test_Result,Pending_Test_Date,Fecha_primer_test_positivo,First_Positive_Test_Result,First_Positive_Change_Date,Creation_Timestamp,Persona_que_creó_el_caso,Fecha_Último_Cambio_Tipo_Seguimiento,Persona_Último_Cambio_Tipo_Seguimiento,Last_Case_Update_Timestamp,Last_Active_Quarantine_From_Date,Quarantine_In_Private_Address,Telecom_Number,Correo_Electrónico,creation_week,dias_desde_creacion
134,100137,NaN,2020-03-29,Idalia Perez Rechia,Rol Único Tributario (RUT),25798817-1,Femenino,2020-06-12,2020-03-30,Negativo,En Cuarentena,No,Sin Información,SAR Renca,Unidad CESFAM Renca,NaN,"11 de diciembre 4138, Renca, Chile",NaN,Renca,NaN,NaN,-33.397804,-70.705830,2020-03-29,Negativo,2020-06-12,NaT,NaN,NaT,2020-04-30 21:52:02.730,NaN,2020-06-18 08:19:20.918,Marcela Tapia,2020-06-16 10:41:51.910,NaT,NaN,56 977012680,NaN,18,49
182,100191,Contacto de Alto Riesgo,2020-04-01,Jael Alegria Carrasco,Rol Único Tributario (RUT),20332776-5,Femenino,2020-06-14,2020-06-16,Positivo,En Cuarentena,No,Leve,CESFAM Bicentenario,Unidad CESFAM Bicentenario,NaN,"Av. Manuel Antonio Caro 1757, Renca, Chile",casa,Renca,NaN,NaN,-33.397961,-70.746129,2020-06-14,Positivo,NaT,2020-06-14,Positivo,2020-06-16 16:46:39.966,2020-04-30 21:52:03.020,NaN,2020-06-17 09:29:47.909,Natalia Hernandez,2020-06-17 19:02:47.484,2020-06-14,True,56 931189747,NaN,18,49
235,100244,Comprobado positivo (PCR positiva),2020-04-03,Olaya Cabrera Labraña,Rol Único Tributario (RUT),13909224-4,Femenino,2020-06-11,2020-06-12,Positivo,En Cuarentena,Sí,Leve,SAR Renca,Unidad CESFAM Bicentenario,NaN,"German Tenderini 1836, Renca, Chile",NaN,Renca,NaN,NaN,-33.398978,-70.739719,2020-04-03,Negativo,NaT,2020-06-11,Positivo,2020-06-12 18:05:10.130,2020-04-30 21:52:03.328,NaN,2020-06-12 10:07:53.277,Paulina Rojas Arriagada,2020-06-17 10:37:33.254,2020-06-11,True,56 999393142,NaN,18,49
261,100270,Comprobado positivo (PCR positiva),2020-05-28,Diego Troncoso Sanchez,Rol Único Tributario (RUT),20355862-7,Masculino,2020-05-28,2020-06-12,Positivo,En Cuarentena,No,Asintomático,CESFAM Bicentenario,Unidad CESFAM Bicentenario,NaN,"Maria Romero 7786, Renca, Chile",NaN,Renca,NaN,NaN,-33.401540,-70.747413,2020-04-06,Negativo,NaT,2020-05-28,Positivo,2020-05-28 11:57:43.383,2020-04-30 21:52:03.441,NaN,2020-06-10 16:16:21.633,José Solar,2020-06-15 11:55:53.303,NaT,NaN,56 940889459,NaN,18,49
312,100325,Contacto de Alto Riesgo,2020-04-06,Maciel Salgado Reyes,Rol Único Tributario (RUT),11391810-1,Femenino,NaT,NaT,Test No Fue Indicado,En Cuarentena,No,Sin Información,SAR Renca,Unidad COVID Central,NaN,"Nueva Platon 5566, Quinta Normal, Chile",NaN,Quinta Normal,NaN,NaN,-33.410832,-70.708943,NaT,NaN,NaT,NaT,NaN,NaT,2020-04-30 21:52:03.660,NaN,2020-05-01 01:33:58.586,NaN,2020-04-30 21:52:16.399,NaT,NaN,56 984945283,NaN,18,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6999,111753,Comprobado positivo (PCR positiva),2020-06-05,Silvana Olea Peña,Rol Único Tributario (RUT),8344020-1,Femenino,2020-06-05,2020-06-06,Positivo,En Cuarentena,No,Moderado,CESFAM Renca,Unidad CESFAM Renca,NaN,"Isabel Riquelme 4537 , Renca, Chile",NaN,Renca,NaN,NaN,NaN,NaN,2020-06-05,Positivo,NaT,2020-06-05,Positivo,2020-06-10 13:06:21.936,2020-06-03 17:04:15.542,Arlyn Montero,2020-06-10 13:08:39.276,Jorge Larrain,2020-06-17 09:35:58.016,2020-06-05,True,56 974630216,NaN,23,15
7001,111804,Contacto de Alto Riesgo,2020-05-31,Maria Olea Peña,Rol Único Tributario (RUT),8344016-3,Femenino,2020-05-31,NaT,No Sabemos,En Cuarentena,No,Leve,CESFAM Renca,Unidad CESFAM R

In [ ]:
#Todos estos debieran salir
pd.pivot_table(df.query('Tipo_de_Seguimiento == "En Cuarentena" and dias_desde_creacion > 14'),index=["creation_week","creation_day","dias_desde_creacion"],values=["ID_del_Caso"],columns=["Tipo_de_Seguimiento"],aggfunc=pd.Series.nunique,margins=True)

ID_del_Caso      
Tipo_de_Seguimiento                            En Cuarentena   All
creation_week creation_day dias_desde_creacion                    
18            2020-04-30   49                             45    45
              2020-05-03   46                              3     3
19            2020-05-07   42                              3     3
              2020-05-08   41                             19    19
              2020-05-09   40                             18    18
              2020-05-10   39                             20    20
20            2020-05-11   38                             18    18
              2020-05-12   37                             42    42
              2020-05-13   36                             28    28
              2020-05-14   35                             36    36
              2020-05-15   34                             65    65
              2020-05-16   33                             49    49
              2020-05-17   32                             38    38
21            2020-05-18   31                             56    56
              2020-05-19   30                             54    54
              2020-05-20   29                            130   130
              2020-05-21   28                            117   117
              2020-05-22   27                             78    78
              2020-05-23   26                             38    38
              2020-05-24   25                             55    55
22            2020-05-25   24                             39    39
              2020-05-26   23                             94    94
              2020-05-27   22                             76    76
              2020-05-28   21                            113   113
              2020-05-29   20                            107   107
              2020-05-30   19                             82    82
              2020-05-31   18                             75    75
23            2020-06-01   17                             57    57
              2020-06-02   16                            126   126
              2020-06-03   15                             96    96
All           NaT                                       1777  1777

#TEST: tipo de caso

In [ ]:
df.Tipo_de_Caso.value_counts(dropna=False)

Contacto de Alto Riesgo                                           4738
Comprobado positivo (PCR positiva)                                3790
NaN                                                               2248
Probable Positivo 2 (Contacto Estrecho de Confirmado Positivo)     175
Sospechoso                                                         149
No Sabemos                                                          51
Probable Positivo 1 (PCR Indeterminado)                              6
Name: Tipo_de_Caso, dtype: int64

In [ ]:
pd.pivot_table(df,index=["Tipo_de_Caso"],values=["ID_del_Caso"],columns=["creation_day"],aggfunc=pd.Series.nunique,margins=True)

ID_del_Caso  ...      
creation_day                                        2020-04-30  ...   All
Tipo_de_Caso                                                    ...      
Comprobado positivo (PCR positiva)                       285.0  ...  3790
Contacto de Alto Riesgo                                  444.0  ...  4738
No Sabemos                                                 NaN  ...    51
Probable Positivo 1 (PCR Indeterminado)                    NaN  ...     6
Probable Positivo 2 (Contacto Estrecho de Confi...         NaN  ...   175
Sospechoso                                                 1.0  ...   149
All                                                      730.0  ...  8909

[7 rows x 51 columns]

In [ ]:
# CRITERIO 1: Filtro de elegibilidad
PCR_pos = (df['First_Positive_Test_Result']=='Positivo')
tipos_caso_list = ["Probable Positivo 1 (PCR Indeterminado)","Comprobado positivo (PCR positiva)"]
tipos_de_caso_elegibles = (df['Tipo_de_Caso'].isin(tipos_caso_list))
tipos_seguimiento_list = ["En Cuarentena"]
seguimiento = df['Tipo_de_Seguimiento'].isin(tipos_seguimiento_list)
primer_filtro = (PCR_pos | tipos_de_caso_elegibles) & seguimiento

In [ ]:
# Primer filtro
df.loc[primer_filtro, 'Filtro1'] = 1

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024 entries, 0 to 11023
Data columns (total 43 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID_del_Caso                             11024 non-null  int64         
 1   Tipo_de_Caso                            8807 non-null   object        
 2   Fecha_de_Inicio_Caso                    11021 non-null  datetime64[ns]
 3   Paciente                                11024 non-null  object        
 4   Tipo_de_Documento                       10998 non-null  object        
 5   Doc_Identidad                           10998 non-null  object        
 6   Género_del_Paciente                     11024 non-null  object        
 7   Fecha_Último_Test                       6722 non-null   datetime64[ns]
 8   Fecha_Últimos_Resultados                5847 non-null   datetime64[ns]
 9   Resultado_Último_Test                   6555 non-n

In [ ]:
df.loc[df.Filtro1 == 1,'Tipo_de_Caso'].value_counts()

Comprobado positivo (PCR positiva)         1159
Contacto de Alto Riesgo                      34
Probable Positivo 1 (PCR Indeterminado)       5
Name: Tipo_de_Caso, dtype: int64

In [ ]:
df.loc[df.Filtro1 == 1,'Tipo_de_Seguimiento'].value_counts()

En Cuarentena    1302
Name: Tipo_de_Seguimiento, dtype: int64

In [ ]:
df.loc[df.Filtro1 == 1,'First_Positive_Test_Result'].value_counts()

Positivo    1294
Name: First_Positive_Test_Result, dtype: int64

In [ ]:
# CRITERIO 2: FECHAS
despues_de = datetime(2020,6,1)
antes_de = datetime(2020,6,17)
recientes = (df['Fecha_de_Inicio_Caso']>=despues_de) & (df['Fecha_de_Inicio_Caso']<=antes_de)
test_pendiente = (df['Pending_Test_Date'].notnull()) & (df['Pending_Test_Date']>=datetime(2020,6,10))
segundo_filtro = recientes | test_pendiente


In [ ]:
df.loc[segundo_filtro, 'Filtro2'] = 1

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024 entries, 0 to 11023
Data columns (total 44 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID_del_Caso                             11024 non-null  int64         
 1   Tipo_de_Caso                            8807 non-null   object        
 2   Fecha_de_Inicio_Caso                    11021 non-null  datetime64[ns]
 3   Paciente                                11024 non-null  object        
 4   Tipo_de_Documento                       10998 non-null  object        
 5   Doc_Identidad                           10998 non-null  object        
 6   Género_del_Paciente                     11024 non-null  object        
 7   Fecha_Último_Test                       6722 non-null   datetime64[ns]
 8   Fecha_Últimos_Resultados                5847 non-null   datetime64[ns]
 9   Resultado_Último_Test                   6555 non-n

In [ ]:
df[(df['Filtro1']==1) & (df['Filtro2']==1)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 929 entries, 897 to 11023
Data columns (total 44 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID_del_Caso                             929 non-null    int64         
 1   Tipo_de_Caso                            837 non-null    object        
 2   Fecha_de_Inicio_Caso                    929 non-null    datetime64[ns]
 3   Paciente                                929 non-null    object        
 4   Tipo_de_Documento                       929 non-null    object        
 5   Doc_Identidad                           929 non-null    object        
 6   Género_del_Paciente                     929 non-null    object        
 7   Fecha_Último_Test                       917 non-null    datetime64[ns]
 8   Fecha_Últimos_Resultados                915 non-null    datetime64[ns]
 9   Resultado_Último_Test                   919 non-nu

In [ ]:
#CRITERO 3 COMUNA DE RESIDENCIA Y CALIDAD DE LAS DIRECCIONES

In [ ]:
residentes = df['Comuna']=='Renca'
con_direccion = df['Dirección']!=", Renca, Chile"
direccion_bien_formada = df['Dirección'].str.contains(r'\d')
georef_correcta = df['Latitud'].notnull()
tercer_filtro = residentes & con_direccion & direccion_bien_formada & georef_correcta

In [ ]:
df.loc[tercer_filtro, 'Filtro3'] = 1

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024 entries, 0 to 11023
Data columns (total 45 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID_del_Caso                             11024 non-null  int64         
 1   Tipo_de_Caso                            8807 non-null   object        
 2   Fecha_de_Inicio_Caso                    11021 non-null  datetime64[ns]
 3   Paciente                                11024 non-null  object        
 4   Tipo_de_Documento                       10998 non-null  object        
 5   Doc_Identidad                           10998 non-null  object        
 6   Género_del_Paciente                     11024 non-null  object        
 7   Fecha_Último_Test                       6722 non-null   datetime64[ns]
 8   Fecha_Últimos_Resultados                5847 non-null   datetime64[ns]
 9   Resultado_Último_Test                   6555 non-n

In [ ]:
df[(df['Filtro1']==1) & (df['Filtro2']==1) & (df['Filtro3']==1)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 739 entries, 897 to 11023
Data columns (total 45 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID_del_Caso                             739 non-null    int64         
 1   Tipo_de_Caso                            675 non-null    object        
 2   Fecha_de_Inicio_Caso                    739 non-null    datetime64[ns]
 3   Paciente                                739 non-null    object        
 4   Tipo_de_Documento                       739 non-null    object        
 5   Doc_Identidad                           739 non-null    object        
 6   Género_del_Paciente                     739 non-null    object        
 7   Fecha_Último_Test                       731 non-null    datetime64[ns]
 8   Fecha_Últimos_Resultados                732 non-null    datetime64[ns]
 9   Resultado_Último_Test                   733 non-nu

In [ ]:
plantilla = df.iloc[:,[3,16,17,21,22,5,36,37,0,27,28,29,25,38,39,40,41,42,43,44]]

In [ ]:
plantilla = plantilla.assign(Carga=1)
plantilla = plantilla.assign(Horainicial='8:30')
plantilla = plantilla.assign(Horafinal='18:00')
plantilla = plantilla.assign(Tiempodeservicio=5)
plantilla = plantilla.assign(Habilidadesrequeridas='')
plantilla = plantilla.assign(HabilidadesOpcionales='')
plantilla = plantilla.assign(Horainicial2='')
plantilla = plantilla.assign(Horafinal2='')
plantilla = plantilla.assign(Carga2='')
plantilla = plantilla.assign(Carga3='')
plantilla = plantilla.assign(Prioridad='')
plantilla = plantilla.assign(SMS='')
plantilla = plantilla.assign(CargaPick='')
plantilla = plantilla.assign(CargaPick2='')
plantilla = plantilla.assign(CargaPick3='')
plantilla = plantilla.assign(Fechaprogramada='')
plantilla = plantilla.assign(Tipodevisita='Entrega Caja Alimentos COVID-19 2020-06-17')

In [ ]:
plantilla = plantilla[[
 'Paciente',
 'Dirección',
 'Carga',
 'Horainicial',
 'Horafinal',
 'Tiempodeservicio',
 'Block_/_Depto_/_Casa',
 'Latitud',
 'Longitud',
 'Doc_Identidad',
 'Habilidadesrequeridas',
 'HabilidadesOpcionales',
 'Paciente',
 'Telecom_Number',
 'Horainicial2',
 'Horafinal2',
 'Carga2',
 'Carga3',
 'Prioridad',
 'SMS',
 'Correo_Electrónico',
 'CargaPick',
 'CargaPick2',
 'CargaPick3',
 'Fechaprogramada',
 'Tipodevisita',
 'ID_del_Caso',
 'First_Positive_Test_Result',
 'First_Positive_Change_Date',
 'Creation_Timestamp',
 'Pending_Test_Date',
 'completed',
 'failed',
 'pending',
 'All',
 'Filtro1',
 'Filtro2',
 'Filtro3' ]]

In [ ]:
nombre_columnas = template.columns

In [ ]:
nombre_columnas = nombre_columnas.append(pd.Index(['ID_del_Caso','First_Positive_Test_Result','First_Positive_Change_Date','Creation_Timestamp','Pending_Test_Date','completed','failed','pending','All','Filtro1','Filtro2', 'Filtro3']))

In [ ]:
plantilla.columns = nombre_columnas

In [ ]:
filtrados = plantilla[(plantilla['Filtro1']==1) & (plantilla['Filtro2']==1) & (plantilla['Filtro3']==1) & (plantilla.All.isna())]

In [ ]:
filtrados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 6757 to 11023
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Título*   Requerido              82 non-null     object        
 1   Dirección completa*   Requerido  82 non-null     object        
 2   Carga                            82 non-null     int64         
 3   Hora inicial                     82 non-null     object        
 4   Hora final                       82 non-null     object        
 5   Tiempo de servicio               82 non-null     int64         
 6   Notas                            17 non-null     object        
 7   Latitud                          82 non-null     float64       
 8   Longitud                         82 non-null     float64       
 9   Id de referencia                 82 non-null     object        
 10  Habilidades requeridas           82 non-null     object   

In [ ]:
path_output = "/content/drive/My Drive/@CORONAVIRUS/CORONAVIRUS COMPARTIDO/Proyecto Entrega Centralizada Cajas/centinela/Plantilla_Centinela_a_Simpliroute_Cruda ({}).xlsx".format(timpestamp_centinela)

     |████████████████████████████████| 143kB 2.8MB/s 


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
import xlsxwriter
writer = pd.ExcelWriter(path_output, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
plantilla.to_excel(writer, sheet_name='todos')
filtrados.to_excel(writer, sheet_name='filtrados')

# Close the Pandas Excel writer and output the Excel file.
writer.save()